In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import shap
from fastshap import KernelExplainer



In [ ]:
# Load data
path = '/trinity/home/atheodosiadou/macro/raw_data/ChIP_comb/ChIP_comb_merged_per_condition/bed_intersect/H3K27ac_EGR1_LPS/H3K27ac_EGR1_LPS_ctrl_overlap_modified_008.bed'
data = pd.read_csv(path, sep="\t", header=None)

feature_names = ["chr_h3k", "start_h3k", "end_h3k", "mvalue_h3k", "avalue_h3k", "pvalue_h3k", "peak_group_h3k",
                 "normalized_density_1_h3k", "normalized_density_2_h3k", 
                 "chr_tf", "start_tf", "end_tf", "mvalue_tf","avalue_tf", "pvalue_tf", "peak_group_tf",
                 "normalized_density_1_tf", "normalized_density_2_tf",
                 "annotation_gene", "distance_to_TSS", "symbol", "gene_name", "overlap"]
data.columns = feature_names

fibrosis_genes_df = pd.read_csv('/trinity/home/atheodosiadou/macro/raw_data/fibrosis/uniprotkb_fibrosis_AND_model_organism_9_2025_06_19.tsv', sep="\t", header = 0)
fibrosis_genes = fibrosis_genes_df['Gene Names'].str.split().explode().unique()
print(fibrosis_genes)
data['fibrosis'] = data['symbol'].isin(fibrosis_genes).astype(int)
print(data)
print(data['fibrosis'].value_counts())

In [ ]:
# Chromosome conversion

def extract_chromosome_number(chrom):
    try:
        return int(chrom[3:])
    except ValueError:
        return {'X': 23, 'Y': 24}.get(chrom[3:], np.nan)

data["chr_h3k"] = data["chr_h3k"].apply(extract_chromosome_number)
data["chr_tf"] = data["chr_tf"].apply(extract_chromosome_number)

# Label mappings
h3k_mapping = {'H3K27ac_LPS.macs3.broad_peaks_filtered_unique': 0, 'H3K27ac_ctrl.macs3.broad_peaks_filtered_unique': 1, 'merged_common' : 2}
tf_mapping = {'EGR1_LPS.macs3_peaks_filtered_unique': 0, 'EGR1_ctrl.macs3_peaks_filtered_unique': 1, 'merged_common': 2, '.': -1}

def annotation_encoder(val):
    if pd.isna(val) or val == ".": return -1
    val = val.lower()
    return {
        "promoter_(<=1kb)": 0, "promoter_(1-2kb)": 1, "promoter_(2-3kb)": 2,
        "intron": 3, "exon": 4, "5'utr": 5, "3'utr": 6,
        "distal_intergenic": 7, "downstream": 8
    }.get(val, -1)

# Feature prep
X = data.drop(["mvalue_h3k", "avalue_h3k", "pvalue_h3k", "normalized_density_1_h3k", "normalized_density_2_h3k",
               "chr_tf", "start_tf", "end_tf", "avalue_tf", "pvalue_tf", "normalized_density_1_tf", "normalized_density_2_tf",
               "distance_to_TSS", "symbol", "gene_name"], axis=1)
X['peak_group_h3k'] = X['peak_group_h3k'].map(h3k_mapping)
X['peak_group_tf'] = X['peak_group_tf'].map(tf_mapping)
X['annotation_gene'] = X['annotation_gene'].apply(annotation_encoder)
X['overlap'] = X['overlap'].apply(lambda x: 1 if int(x) > 0 else 0)
y = data["mvalue_h3k"]
print(data)

In [ ]:
# #keep only the 2 minority classes(mvalue_h3k == 1 and mvalue_h3k == -1)
# data = data[(data['mvalue_h3k'] == 1) | (data['mvalue_h3k'] == -1)]

In [ ]:
# Split before SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
# Identify columns with "." and replace with NaN
for col in X.columns:
    try:
        X_train[col] = pd.to_numeric(X_train[col])  # Try to convert to numeric
        X_test[col] = pd.to_numeric(X_test[col])
    except ValueError:
        print(f"Column {col} contains non-numeric values. Replacing '.' with NaN.")
        X_train[col] = X_train[col].replace('.', np.nan)  # Replace "." with NaN
        X_test[col] = X_test[col].replace('.', np.nan)
        X_train[col] = pd.to_numeric(X_train[col])  # Convert to numeric after replacement
        X_test[col] = pd.to_numeric(X_test[col])

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [ ]:
# Stratified K-Fold
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
# Dummy classifier
dummy = DummyClassifier(strategy='most_frequent')
dummy_scores = cross_val_score(dummy, X_train, y_train, cv=skf, scoring='f1_weighted')
print("\nBaseline Dummy Classifier F1 (Weighted) CV Scores:", dummy_scores)
print("Mean F1:", np.mean(dummy_scores))

In [ ]:
# Random Forest with SMOTE pipeline
rf_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])
param_grid_rf = {
    'clf__n_estimators': [500, 1000],
    'clf__max_features': ['sqrt', 'log2'],
    'clf__max_depth': [None, 10, 20],
    'clf__criterion': ['gini', 'entropy']
}
rf_search = RandomizedSearchCV(rf_pipeline, param_grid_rf, n_iter=5, scoring='f1_weighted', n_jobs=-1, cv=skf)
rf_search.fit(X_train, y_train)
print("\nBest RF params:", rf_search.best_params_)
y_pred_rf = rf_search.best_estimator_.predict(X_test)
print("\nRandom Forest Classification Report:")
report = classification_report(y_test, y_pred_rf)
print(report)

# Save to a text file
with open("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/classification_report_rf_merged_manorm_008.txt", "w") as f:
    f.write(report)



# ])
# param_grid = {
#     'clf__n_estimators': [500, 1000],
#     'clf__max_features': ['sqrt', 'log2'],
#     'clf__max_depth': [None, 10, 20],
#     'clf__criterion': ['gini', 'entropy'],
#     'clf__min_samples_leaf': [1, 2, 4]
# }
# grid = GridSearchCV(rf_pipeline, param_grid, cv=skf, scoring='f1_weighted', n_jobs=-1)
# grid.fit(X_train, y_train)
# print("\nBest RF params:", grid.best_params_)
# y_pred_rf = grid.best_estimator_.predict(X_test)
# print("\nRandom Forest Classification Report:")
# print(classification_report(y_test, y_pred_rf))

In [ ]:
# Logistic Regression pipeline
lr_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('clf', LogisticRegression(penalty='l2', solver='lbfgs', max_iter=10000, class_weight='balanced', random_state=42))
])

# Logistic Regression hyperparameter tuning
param_grid_lr = {
    'clf__C': [0.01, 0.1, 1]
}

lr_search = RandomizedSearchCV(lr_pipeline, param_grid_lr, n_iter=3, scoring='f1_weighted', n_jobs=-1, cv=skf)
lr_search.fit(X_train, y_train)
print("\nBest LR params:", lr_search.best_params_)
y_pred_lr = lr_search.best_estimator_.predict(X_test)

print("\nLogistic Regression Report:")


report = classification_report(y_test, y_pred_lr, zero_division=0)
print(report)

# Save to a text file
with open("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/classification_report_lr_merged_manorm_008.txt", "w") as f:
    f.write(report)

In [ ]:
# XGBoost pipeline
xgb_pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('clf', XGBClassifier(eval_metric='mlogloss', random_state=42))
])

# XGBoost hyperparameter tuning
param_grid_xgb = {
    'clf__n_estimators': [100, 200, 500],
    'clf__max_depth': [3, 5, 7],
    'clf__learning_rate': [0.01, 0.1, 0.2],
    'clf__gamma': [0, 0.1, 0.2]
}

xgb_search = RandomizedSearchCV(xgb_pipeline, param_grid_xgb,  n_iter=5, scoring='f1_weighted', n_jobs=-1, cv=skf)
xgb_search.fit(X_train, y_train)
print("\nBest XGB params:", xgb_search.best_params_)
y_pred_xgb = xgb_search.best_estimator_.predict(X_test)

report = classification_report(y_test, y_pred_xgb)
print(report)

# Save to a text file
with open("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/classification_report_xgb_merged_manorm_008.txt", "w") as f:
    f.write(report)

In [ ]:
# # SVM pipeline
# svm_pipeline = ImbPipeline([
#     ('smote', SMOTE(random_state=42)),
#     ('clf', SVC(kernel='linear', class_weight='balanced', probability=False, random_state=42))
# ])

# # # SVM hyperparameter tuning
# # param_grid_svm = {
# #     'clf__C': [0.1, 1, 10],
# #     'clf__gamma': [0.001, 0.01, 0.1]
# # }
# # svm_search = RandomizedSearchCV(svm_pipeline, param_grid_svm, n_iter=5, scoring='f1_weighted', n_jobs=-1, cv=skf)
# # svm_search.fit(X_train, y_train)
# # print("\nBest SVM params:", svm_search.best_params_)

# # y_pred_svm = svm_search.best_estimator_.predict(X_test)


# # svm_pipeline.fit(X_train, y_train)
# # print("\nBest SVM params:", svm_pipeline.best_params_)

# # y_pred_svm = svm_pipeline.best_estimator_.predict(X_test)


# # svm_scores = cross_val_score(svm_pipeline, X_train, y_train, cv=skf, scoring='f1_weighted')
# # print("\nSVM CV F1 Scores:", svm_scores)
# svm_pipeline.fit(X_train, y_train)
# y_pred_svm = svm_pipeline.predict(X_test)
# print("\nSVM Report:")

# report = classification_report(y_test, y_pred_svm, zero_division=0) #  Handle zero division
# print(report)

# # Save to a text file
# with open("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/classification_report_svm_merged_manorm_008.txt", "w") as f:
#     f.write(report)

In [ ]:
# SHAP values for best RF
sample_X = shap.sample(pd.DataFrame(X_test, columns=X.columns), 100, random_state=42)
explainer = shap.TreeExplainer(rf_search.best_estimator_.named_steps['clf'])
shap_values = explainer(sample_X)
shap.summary_plot(shap_values, features=sample_X, feature_names=[sample_X.columns[i] for i in range(sample_X.shape[1])], show=False)
plt.title("SHAP Summary Plot for Random Forest Model - H3K27ac_EGR1_LPS")
plt.tight_layout()
plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/shap_summary_merged_manorm_008.png", dpi=150)
plt.show()

In [ ]:
# SHAP values for best XGB
sample_X = shap.sample(pd.DataFrame(X_test, columns=X.columns), 100, random_state=42)
explainer = shap.Explainer(xgb_search.best_estimator_.named_steps['clf'])
shap_values = explainer(sample_X)
shap.summary_plot(shap_values, features=sample_X, feature_names=[sample_X.columns[i] for i in range(sample_X.shape[1])], show=False)
plt.title("SHAP Summary Plot for XGBoost Model - H3K27ac_EGR1_LPS")
plt.tight_layout()
plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/shap_summary_xgb_merged_manorm_008.png", dpi=150)
plt.show()

In [ ]:
# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='Blues')
plt.title("Random Forest Confusion Matrix - H3K27ac_EGR1_LPS")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/confusion_matrix_rf_merged_manorm_008.png")
plt.show()

In [ ]:
# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred_xgb), annot=True, fmt='d', cmap='Blues')
plt.title("XGBoost Confusion Matrix - H3K27ac_EGR1_LPS")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/confusion_matrix_xgb_merged_manorm_008.png")
plt.show()

In [ ]:
# --- Append this block at the end of your script ---

# == Analyzing most important peaks for mvalue = 1 and overlap = 1 ==

# Recreate column names from X after preprocessing
column_names = X.columns.tolist()
print(column_names)

# Convert X_test back to DataFrame
X_test_df = pd.DataFrame(X_test, columns=column_names)
# print(X_test_df.head())
X_test_df = X_test_df.reset_index(drop=True)
# print(X_test_df)
y_test_reset = y_test.reset_index(drop=True)
# print(y_test_reset)


# Merge with original test rows for context
original_test_data = data.iloc[y_test.index].reset_index(drop=True) #this will keep the original rows intact and the original column names
# print(original_test_data.head())

# Filter for rows where mvalue == 1 and overlap == 1
filtered_idx = (y_test_reset == 2) & (X_test_df['overlap'] == 1) #false for the cases where mvalue == 0 or 1, and true for the cases where mvalue == 2
# print(filtered_idx)
X_filtered = X_test_df[filtered_idx]
# print(X_filtered)
original_filtered = original_test_data[filtered_idx].copy() #uses the same rows as X_filtered but keeps the original columns
# print(original_filtered)

# SHAP explainer from RF already defined: 'explainer'
# Compute SHAP values on filtered rows
print("\nComputing SHAP values for mvalue=2 and overlap=1 subset...")

shap_values_filtered = explainer(X_filtered)
# print(shap_values_filtered)
shap_vals_array = shap_values_filtered.values
# print(shap_vals_array)
print(f"SHAP values shape: {shap_vals_array.shape}")

# Compute mean absolute SHAP values across filtered samples
mean_abs_shap = np.abs(shap_vals_array[:, :, 2]).mean(axis=0)
print(mean_abs_shap)  
shap_feature_importance = pd.DataFrame({
    'Feature': column_names,
    'Mean_Abs_SHAP': mean_abs_shap
}).sort_values(by='Mean_Abs_SHAP', ascending=False)

# Output top features
print("\nTop features contributing to mvalue = 2 and overlap = 1:")
print(shap_feature_importance.head(10))

# Sample-level total contribution
shap_total_contrib = np.sum(np.abs(shap_vals_array[:, :, 2]), axis=1)
print(f"Total SHAP contributions shape: {shap_total_contrib.shape}")
print(f"Total SHAP contributions: {shap_total_contrib}")
original_filtered['Total_SHAP'] = shap_total_contrib

# Sort and show top peaks
top_peaks = original_filtered.sort_values(by='Total_SHAP', ascending=False).head(100)
print("\nTop peaks contributing to mvalue = 2 and overlap = 1:")
print(top_peaks[['chr_h3k', 'start_h3k', 'end_h3k', 'symbol', 'annotation_gene','fibrosis', 'Total_SHAP']])

# Save to CSV
top_peaks.to_csv("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/fibrosis/H3K27ac_EGR1_LPS/top_peaks_mval1_overlap1_merged_manorm_008.csv", sep="\t",  index=False)

In [ ]:
# # 4. Feature Selection using Permutation Importance
# smote = SMOTE(random_state=42)
# X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
# print(f"After SMOTE, X_train shape: {X_train_smote.shape}, y_train shape: {y_train_smote.shape}")
# y_train_smote.hist()
# model_for_feature_selection = RandomForestClassifier(n_estimators=1000, random_state=42)
# model_for_feature_selection.fit(X_train_smote, y_train_smote)

# # Select features based on importance
# sfm = SelectFromModel(model_for_feature_selection, threshold=0.0001)  
# sfm.fit(X_train_smote, y_train_smote)
# X_train_selected = sfm.transform(X_train_smote)
# X_test_selected = sfm.transform(X_test)

# # Get selected feature names
# selected_feature_indices = sfm.get_support(indices=True)
# selected_feature_names = [X.columns[i] for i in selected_feature_indices]


In [ ]:
# from sklearn.model_selection import GridSearchCV

# # Define parameter grid for Random Forest
# param_grid = {
#     'n_estimators': [500,1000],  # Number of trees in the forest
#     'max_depth': [None, 10, 20],
#     # 'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2'],
#     # 'bootstrap': [True, False],
#     'criterion': ['gini', 'entropy'],
#     # 'max_samples': [None, 0.5, 0.75],  # Optional, for sub-sampling
#     # 'max_leaf_nodes': [None, 10, 20, 30],  # Optional, for limiting leaf nodes
#     # 'min_weight_fraction_leaf': [0.1, 0.2],  # Optional, for handling class imbalance
#     # 'ccp_alpha': [0.0, 0.01, 0.1],  # Optional, for pruning
#     # 'class_weight': ['balanced', 'balanced_subsample']  # Optional, for handling class imbalance
# }

# # GridSearchCV
# rf = RandomForestClassifier(random_state=42)
# grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1, scoring='f1_macro')
# grid_search.fit(X_train_selected, y_train_smote)  # Use balanced y after SMOTE

# print("Best parameters found:", grid_search.best_params_)
# print("Best cross-validated F1 score:", grid_search.best_score_)

# # Use the best estimator for further analysis
# model = grid_search.best_estimator_


In [ ]:
# # 6. Model Evaluation
# y_pred = model.predict(X_test_selected)
# balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
# print(f"Balanced Accuracy: {balanced_accuracy}")
# # accuracy = accuracy_score(y_test, y_pred)
# # print(f"Accuracy: {accuracy}")
# print(classification_report(y_test, y_pred))

# cm = confusion_matrix(y_test, y_pred)
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
# plt.xlabel("Predicted")
# plt.ylabel("Actual")
# plt.title("Confusion Matrix")
# plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/H3K27ac_EGR1_LPS/confusion_matrix_minority_no_start_end.png")
# plt.show()

In [ ]:
# # 7. Feature Importance Plot (using selected features)
# result = permutation_importance(model, X_test_selected, y_test, n_repeats=10, random_state=42)
# importances = result.importances_mean
# indices = np.argsort(importances)[::-1]
# print(indices)
# print("Selected feature names:", selected_feature_names)
# print("Feature importances (selected features):")

# plt.figure(figsize=(12, 10))
# plt.title("Feature Importances (Selected Features)")
# plt.bar(range(X_train_selected.shape[1]), importances[indices], align="center")
# plt.xticks(range(X_train_selected.shape[1]), [selected_feature_names[i] for i in indices], rotation=90)
# plt.xlabel("Feature")
# plt.ylabel("Importance")
# plt.tight_layout()
# plt.savefig("/trinity/home/atheodosiadou/macro/results/ChIP_comb/models/H3K27ac_EGR1_LPS/feature_importances_selected_0001_minority.png")
# plt.show()